In [25]:
#function tokenisasi
def tokenisasi(text):
    tokens = text.split(" ")
    return tokens

#function stemming
def stemming(text):
    from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
    # create stemmer
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    # stemming process
    output = stemmer.stem(text)
    return output

#function stemming sentence
def stemming_sentence(text):
    output = ""
    for token in tokenisasi(text):
        output = output + stemming(token) + " "
    return output[:-1]

doc_dict_raw = {}
doc_dict_raw['doc1']="pengembangan sistem informasi penjadwalan"
doc_dict_raw['doc2']="pengembangan model analisis sentimen berita"
doc_dict_raw['doc3'] = "analisis sistem input output"
doc_dict_raw['doc4'] = "pengembangan sistem informasi akademik universitas"
doc_dict_raw['doc5'] = "pengembangan sistem cari berita ekonomi"
doc_dict_raw['doc6'] = "analisis sistem neraca nasional"
doc_dict_raw['doc7'] = "pengembangan sistem informasi layanan statistik"
doc_dict_raw['doc8'] = "pengembangan sistem pencarian skripsi di universitas"
doc_dict_raw['doc9'] = "analisis sentimen publik terhadap pemerintah"
doc_dict_raw['doc10'] = "pengembangan model klasifikasi sentimen berita"

doc_dict ={}
for doc_id,doc in doc_dict_raw.items():
    doc_dict[doc_id]=stemming_sentence(doc)
print(doc_dict)


{'doc1': 'kembang sistem informasi jadwal', 'doc2': 'kembang model analisis sentimen berita', 'doc3': 'analisis sistem input output', 'doc4': 'kembang sistem informasi akademik universitas', 'doc5': 'kembang sistem cari berita ekonomi', 'doc6': 'analisis sistem neraca nasional', 'doc7': 'kembang sistem informasi layan statistik', 'doc8': 'kembang sistem cari skripsi di universitas', 'doc9': 'analisis sentimen publik hadap perintah', 'doc10': 'kembang model klasifikasi sentimen berita'}


In [33]:
vocab = []
inverted_index= {}
for doc_id, doc in doc_dict.items():
    for token in tokenisasi(doc):
        
        if token not in vocab:
            vocab.append(token)
            inverted_index[token] = []
        if token in inverted_index:
            if doc_id not in inverted_index[token]:
                inverted_index[token].append(doc_id)
#print(vocab)

print(doc)

kembang model klasifikasi sentimen berita


In [5]:
print(len(doc_dict))

10


In [6]:
query = "Sistem pengembangan universitas di dalam kehidupan sehari-hari"
def termFrequency(vocab, query):
    tf_query = {}
    for word in vocab:
        tf_query[word] = query.count(word)
    return tf_query
tf_query = termFrequency(vocab, query)
print(tf_query)

{'kembang': 0, 'sistem': 0, 'informasi': 0, 'jadwal': 0, 'model': 0, 'analisis': 0, 'sentimen': 0, 'berita': 0, 'input': 0, 'output': 0, 'akademik': 0, 'universitas': 1, 'cari': 0, 'ekonomi': 0, 'neraca': 0, 'nasional': 0, 'layan': 0, 'statistik': 0, 'skripsi': 0, 'di': 1, 'publik': 0, 'hadap': 0, 'perintah': 0, 'klasifikasi': 0}


In [7]:
import numpy as np

#menghitung df yang merupakan freq kata pada suatu dokumen 
def wordDocFre(vocab,doc_dict):
    df={}
    for word in vocab:
        frq = 0
        for doc in doc_dict.values():
            if word in tokenisasi(doc):
                frq=frq+1
        df[word]=frq 
    return df

print(wordDocFre(vocab,doc_dict))

def inverseDocFre(vocab,doc_fre,length):
    idf={}
    for word in vocab:
        idf[word]=idf[word]=1+np.log10((length+1)/(doc_fre[word]+1))
    return idf

print(inverseDocFre(vocab,wordDocFre(vocab,doc_dict),len(doc_dict)))
idf = inverseDocFre(vocab,wordDocFre(vocab,doc_dict),len(doc_dict))

# Term - Query Matrix
TQ = np.zeros((len(vocab), 1)) #hanya 1 query
length=len(doc_dict)
#idf = inverseDocFre(vocab,doc_dict,length)
for word in vocab:
    ind1 = vocab.index(word)
    TQ[ind1][0] = tf_query[word]*idf[word]
print(TQ)

def termFrequencyInDoc(vocab,doc_dict):
    tf_docs = {}
    for doc_id in doc_dict.keys():
        tf_docs[doc_id]={}
    
    for word in vocab:
        for doc_id, doc in doc_dict.items():
            tf_docs[doc_id][word]=doc.count(word)
    return tf_docs

def tfidf(vocab,tf,idf_scr,doc_dict):
    tf_idf_scr={}
    for doc_id in doc_dict.keys():
        tf_idf_scr[doc_id]={}
    for word in vocab:
        for doc_id, doc in doc_dict.items():
            tf_idf_scr[doc_id][word] = tf[doc_id][word]*idf_scr[word]
    return tf_idf_scr
#tem document
tf_idf = tfidf(vocab, termFrequencyInDoc(vocab,doc_dict),inverseDocFre(vocab,wordDocFre(vocab,doc_dict),len(doc_dict)),doc_dict)
TD = np.zeros((len(vocab),len(doc_dict)))
for word in vocab:
    for doc_id, doc in tf_idf.items():
        ind1=vocab.index(word)
        ind2=list(tf_idf.keys()).index(doc_id)
        TD[ind1][ind2]=tf_idf[doc_id][word]
print(TD)

{'kembang': 7, 'sistem': 7, 'informasi': 3, 'jadwal': 1, 'model': 2, 'analisis': 4, 'sentimen': 3, 'berita': 3, 'input': 1, 'output': 1, 'akademik': 1, 'universitas': 2, 'cari': 2, 'ekonomi': 1, 'neraca': 1, 'nasional': 1, 'layan': 1, 'statistik': 1, 'skripsi': 1, 'di': 1, 'publik': 1, 'hadap': 1, 'perintah': 1, 'klasifikasi': 1}
{'kembang': 1.1383026981662814, 'sistem': 1.1383026981662814, 'informasi': 1.4393326938302626, 'jadwal': 1.7403626894942439, 'model': 1.5642714304385625, 'analisis': 1.3424226808222062, 'sentimen': 1.4393326938302626, 'berita': 1.4393326938302626, 'input': 1.7403626894942439, 'output': 1.7403626894942439, 'akademik': 1.7403626894942439, 'universitas': 1.5642714304385625, 'cari': 1.5642714304385625, 'ekonomi': 1.7403626894942439, 'neraca': 1.7403626894942439, 'nasional': 1.7403626894942439, 'layan': 1.7403626894942439, 'statistik': 1.7403626894942439, 'skripsi': 1.7403626894942439, 'di': 1.7403626894942439, 'publik': 1.7403626894942439, 'hadap': 1.7403626894942

In [8]:
import math
def cosine_sim(vec1,vec2):
    vec1=list(vec1)
    vec2=list(vec2)
    dot_prod=0
    for i, v in enumerate(vec1):
        dot_prod+=v*vec2[i]
    mag_1=math.sqrt(sum([x**2 for x in vec1]))
    mag_2=math.sqrt(sum([x**2 for x in vec2]))
    return dot_prod/(mag_1 * mag_2)

print(cosine_sim(TQ[:, 0], TD[:, 0])) #query & doc1
print(cosine_sim(TQ[:, 0], TD[:, 1])) #query & doc2
print(cosine_sim(TQ[:, 0], TD[:, 2])) #query & doc3
print(cosine_sim(TQ[:, 0], TD[:, 3])) #query & doc3
print(cosine_sim(TQ[:, 0], TD[:, 4])) #query & doc3
print(cosine_sim(TQ[:, 0], TD[:, 5])) #query & doc3
print(cosine_sim(TQ[:, 0], TD[:, 6])) #query & doc3
print(cosine_sim(TQ[:, 0], TD[:, 7])) #query & doc3
print(cosine_sim(TQ[:, 0], TD[:, 8])) #query & doc3


0.0
0.0
0.0
0.328400348945259
0.0
0.0
0.0
0.635865623181743
0.0


In [38]:
TD[:, 0]

array([1.1383027 , 1.1383027 , 1.43933269, 1.74036269, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ])

In [9]:
from collections import OrderedDict
def exact_top_k(doc_dict, TD, q, k):
    relevance_scores = {}
    i = 0
    for doc_id in doc_dict.keys():
        relevance_scores[doc_id] = cosine_sim(q, TD[:, i])
        i = i + 1
    sorted_value = OrderedDict(sorted(relevance_scores.items(), key=lambda x: x[1], reverse = True))
    top_k = {j: sorted_value[j] for j in list(sorted_value)[:k]}
    return top_k
top_2 = exact_top_k(doc_dict, TD, TQ[:, 0], 2)
#misal k = 3
top_3=exact_top_k(doc_dict,TD,TQ[:,0],3)
top_2=exact_top_k(doc_dict,TD,TQ[:,0],2)
print(top_2)

{'doc8': 0.635865623181743, 'doc4': 0.328400348945259}


# Index Elimination

In [10]:
def index_elim_simple(query,doc_dict):
    remove_list=[]
    for doc_id, doc in doc_dict.items():
        n=0
        for word in tokenisasi(query):
            if stemming(word) in doc:
                n=n+1
        if n==0:
                remove_list.append(doc_id)
    for key in remove_list:
            del doc_dict[key]
    return doc_dict

In [11]:
query ="sistem informasi statistik"
doc_dict=index_elim_simple(query,doc_dict)

In [12]:
def elim_query(query,idf_dict, idf_score):
    for term in tokenisasi(query):
        if idf_dict[stemming(term)]<idf_score:
            query=query.replace(term+"","")
            query=query.replace(term,"")
    return query

In [13]:
query="sistem informasi statistik"
query=elim_query(query,idf,1.5)
print(query)

  statistik


# Champion List

In [14]:
def create_championlist(inverted_index,tf_idf,r):
    champion_list={}
    for term in inverted_index.keys():
        weight_scores={}
        for doc_id,tf in tf_idf.items():
            if tf_idf[doc_id][term]!=0:
                weight_scores[doc_id] = tf_idf[doc_id][term]
        sorted_value= OrderedDict(sorted(weight_scores.items(),key=lambda x:x[1],reverse=True))
        top_r={j: sorted_value[j] for j in list(sorted_value)[:r]}
        champion_list[term]=list(top_r.keys())
    return champion_list

Kemudian panggil fungsi di atas untuk mendapatkan champion list untuk r tertentu, misalnya r=2.
Bandingkan isi champion list dan inverted index yang telah dibuat sebelumnya.

In [15]:
r=2
create_championlist(inverted_index,tf_idf,r)

{'kembang': ['doc1', 'doc2'],
 'sistem': ['doc1', 'doc3'],
 'informasi': ['doc1', 'doc4'],
 'jadwal': ['doc1'],
 'model': ['doc2', 'doc10'],
 'analisis': ['doc2', 'doc3'],
 'sentimen': ['doc2', 'doc9'],
 'berita': ['doc2', 'doc5'],
 'input': ['doc3'],
 'output': ['doc3'],
 'akademik': ['doc4'],
 'universitas': ['doc4', 'doc8'],
 'cari': ['doc5', 'doc8'],
 'ekonomi': ['doc5'],
 'neraca': ['doc6'],
 'nasional': ['doc6'],
 'layan': ['doc7'],
 'statistik': ['doc7'],
 'skripsi': ['doc8'],
 'di': ['doc8'],
 'publik': ['doc9'],
 'hadap': ['doc9'],
 'perintah': ['doc9'],
 'klasifikasi': ['doc10']}